In [1]:
#execute just the first time to move scripts and dataset from input to output
import sys
from shutil import copytree, copyfile
sys.path.append( "/kaggle/input/cityscapes" )

src_datasets='/kaggle/input/cityscapes/datasets/'
dst_datasets='/kaggle/working/datasets/'
copytree(src_datasets, dst_datasets)

src_datasets='/kaggle/input/cityscapes/Cityscapes_v2.py'
dst_datasets='/kaggle/working/datasets/Cityscapes_v2.py'
copyfile(src_datasets, dst_datasets)

src_datasets='/kaggle/input/cityscapes/deeplab_resnet_pretrained_imagenet.pth'
dst_datasets='/kaggle/working/deeplab_resnet_pretrained_imagenet.pth'
copyfile(src_datasets, dst_datasets)

src_models='/kaggle/input/cityscapes/models'
dst_models='/kaggle/working/models/'
copytree(src_models, dst_models)

copyfile(src = "/kaggle/input/cityscapes/utils.py", dst = "/kaggle/working/utils.py")

'/kaggle/working/utils.py'

In [2]:
#this one you need it always
sys.path.append('/kaggle/input/cityscapes/datasets/') 

In [3]:
# TODO: Define here your training and validation loops.

import torch.nn.functional as F
from Cityscapes_v2 import CityScapes
from torch.utils.data import DataLoader
import torch
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
from models.deeplabv2.deeplabv2 import Bottleneck
from models.deeplabv2.deeplabv2 import ResNetMulti
from models.deeplabv2.deeplabv2 import get_deeplab_v2
from utils import poly_lr_scheduler

def mIoU(pred_mask, mask, smooth=1e-10, n_classes=19):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                #print("true class size",true_class.shape)
                #print("true label size",true_label.shape)
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union + smooth)
                iou_per_class.append(iou)
        return np.nanmean(iou_per_class)
    
def pixel_accuracy(output, mask):
    with torch.no_grad():
        output = torch.argmax(F.softmax(output, dim=1), dim=1)
        correct = torch.eq(output, mask).int()
        accuracy = float(correct.sum()) / float(correct.numel())
    return accuracy

def convert_tensor_to_image(tensor):
    image = tensor.permute(1, 2, 0)
    return image

def train(model, optimizer, train_loader, criterion):
    model.train()
    running_loss = 0.0
    total = 0
    iou_score=0
    accuracy=0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs = inputs.float()
        targets = targets.squeeze(dim=1)
        
        #Compute prediction and loss
        outputs,_,_ = model(inputs)
        loss = loss_fn(outputs.to(dtype=torch.float32), targets.to(dtype=torch.int64))
        iou_score += mIoU(outputs.to(device), targets.to(device))
        accuracy += pixel_accuracy(outputs.to(device), targets.to(device))
        
        #BackPropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        
#     before_lr = optimizer.param_groups[0]["lr"]
#     scheduler.step()
#     after_lr = optimizer.param_groups[0]["lr"]

    train_loss = running_loss / len(train_loader)
    iou_score = iou_score / len(train_loader)
    accuracy = accuracy / len(train_loader)
    return train_loss,iou_score,accuracy

# Test loop
def test(model, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    iou_score=0.0
    accuracy=0.0
    with torch.no_grad():
        for batch_idx,(inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            inputs = inputs.float()
            targets = targets.int()
            
            #Compute prediction and loss
            outputs = model(inputs)
            loss = loss_fn(outputs.to(dtype=torch.float32), targets.squeeze(dim=1).to(dtype=torch.int64))
            iou_score += mIoU(outputs.to(device), targets.to(device))
            accuracy += pixel_accuracy(outputs.to(device), targets.to(device))
            test_loss += loss.item()
            
    test_loss = test_loss / len(test_loader)
    iou_score = iou_score / len(test_loader)
    accuracy = accuracy / len(test_loader)
    #test_accuracy = 100. * correct / total
    return test_loss,iou_score,accuracy

dataset_path='/kaggle/input/cityscapes/Cityscapes/Cityscapes/Cityspaces/'
image_train=dataset_path+'images/train'
sem_map_train=dataset_path+'gtFine/train'
image_val=dataset_path+'images/val'
sem_map_val=dataset_path+'gtFine/val'

resize_transform = transforms.Resize(interpolation=transforms.InterpolationMode.NEAREST_EXACT,size = (512,1024))

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [4]:
# Define dataset and loader
cityscapes_train = CityScapes(annotations_dir=sem_map_train, images_dir=image_train,transform=resize_transform)
cityscapes_val = CityScapes(annotations_dir=sem_map_val, images_dir=image_val,transform=resize_transform)

bs = 5
train_loader = DataLoader(cityscapes_train, batch_size=bs, shuffle=False)
val_loader = DataLoader(cityscapes_val, batch_size=bs, shuffle=False)

In [7]:
print(f'training dataset contains {cityscapes_train.__len__()} images')
print(f'validation dataset contains {cityscapes_val.__len__()} images')

index = 448
# print('Image path: \n'f'{cityscapes_train.map_index_to_image[index]}')
# print('Map path: \n'f'{cityscapes_train.map_index_to_annotation[index]}')
image, sem_map = cityscapes_train.__getitem__(index)
sem_map.size()
fig, axes = plt.subplots(2, 1)
image_transpose = convert_tensor_to_image(image)
map_transpose = convert_tensor_to_image(sem_map)
axes[0].imshow(image_transpose)
axes[1].imshow(map_transpose, cmap='Blues')
plt.show()

training dataset contains 1572 images
validation dataset contains 500 images


In [6]:
#pretrained model
# Define the model and move it to the device
model = ResNetMulti(Bottleneck, [3, 4, 23, 3], num_classes=19)
model.to(device)
start_lr = 0.01
epoch = 0 #to initialize the lr
optimizer = torch.optim.Adam(model.parameters(), lr=start_lr)
l_rate = poly_lr_scheduler(optimizer, init_lr=start_lr , iter=epoch, lr_decay_iter=1, max_iter=50, power=0.9)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=255)

# Freeze parameters except ClassifierModule
for param in model.get_1x_lr_params_no_scale():
    param.requires_grad = False

In [7]:
for epoch in range(50):
    l_rate = poly_lr_scheduler(optimizer, init_lr=start_lr , iter=epoch, lr_decay_iter=1, max_iter=50, power=0.9)
    print(l_rate)
    print(optimizer)

0.01
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)
0.009819818665965752
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.009819818665965752
    maximize: False
    weight_decay: 0
)
0.009639269212585509
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.009639269212585509
    maximize: False
    weight_decay: 0
)
0.009458343193786321
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.009458343193786321
    maximize: False
    weight_decay: 0
)
0.00

In [10]:
from torchinfo import summary
summary(model, input_size=(bs,3,512,1024), col_names=["output_size", "num_params", "trainable"])

Layer (type:depth-idx)                   Output Shape              Param #                   Trainable
ResNetMulti                              [5, 19, 512, 1024]        --                        Partial
├─Conv2d: 1-1                            [5, 64, 256, 512]         (9,408)                   False
├─BatchNorm2d: 1-2                       [5, 64, 256, 512]         (128)                     False
├─ReLU: 1-3                              [5, 64, 256, 512]         --                        --
├─MaxPool2d: 1-4                         [5, 64, 129, 257]         --                        --
├─Sequential: 1-5                        [5, 256, 129, 257]        --                        False
│    └─Bottleneck: 2-1                   [5, 256, 129, 257]        --                        False
│    │    └─Conv2d: 3-1                  [5, 64, 129, 257]         (4,096)                   False
│    │    └─BatchNorm2d: 3-2             [5, 64, 129, 257]         (128)                     False
│    │    

In [ ]:
!pip install -U fvcore

In [ ]:
# FLOPS
from fvcore.nn import FlopCountAnalysis, flop_count_table

plot_loader = DataLoader(cityscapes_train, batch_size=1, shuffle=True)
(input,output) = next(iter(plot_loader))
height = 512
width = 1024

flops = FlopCountAnalysis(model, input.to(device,dtype=torch.float32))
print(flop_count_table(flops))

In [ ]:
# Latency and FPS
import time 
import numpy as np
net = model
net.eval()
fps_loader = DataLoader(cityscapes_train, batch_size=1, shuffle=True)
(inputs, annotations) = next(iter(fps_loader))
inputs=inputs.to(device,dtype=torch.float32)
iterations=1000
latency=np.empty(0)
FPS=np.empty(0)
for i in range(iterations):
    start=time.time()
    output=net(inputs)
    end=time.time()
    latency_i=end-start
    #print(latency_i)
    latency=np.append(latency,latency_i)
    FPS_i=float(1/latency_i)
    FPS=np.append(FPS,FPS_i)
meanLatency=np.mean(latency)
stdLatency=np.std(latency)
meanFPS=np.mean(FPS)
stdFPS=np.std(FPS)


In [ ]:
print (f"mean latency: {meanLatency} seconds")
print(f"std latency: {stdLatency} seconds")
print (f"mean FPS: {meanFPS} fps")
print(f"std FPS: {stdFPS} fps")

In [11]:
train_iou_list=[]
train_acc_list=[]
train_loss_list=[]
test_iou_list=[]
test_acc_list=[]
test_loss_list=[]

In [12]:
# Train the model

epoch_beginning=0
epochs = 50

for epoch in range(epoch_beginning,epochs):
    n_epoch = epoch+1
    l_rate = poly_lr_scheduler(optimizer, init_lr=start_lr , iter=epoch, lr_decay_iter=1, max_iter=50, power=0.9)
    train_loss,train_iou,train_acc=train(model, optimizer, train_loader, loss_fn)
    train_iou_list.append(train_iou)
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    
    #save model
    if n_epoch%5 == 0 or n_epoch==epochs:
        model_name = f"deeplabv2_bs{bs}_epoch{n_epoch}_weights.pth"
        torch.save(model.state_dict(), model_name)
        
    test_loss,test_iou,test_acc = test(model, val_loader, loss_fn)
    test_iou_list.append(test_iou)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)
    print(f"Epoch: {n_epoch}")
    print(f"- Train Acc: {train_acc:.3f}")
    print(f"- Train mIou: {train_iou:.3f}\n")
    print(f"- Test Acc: {test_acc:.3f}")
    print(f"- Test Loss: {test_loss:.3f}")
    print(f"- Test mIoU: {test_iou:.3f}")
    print("__________________________")

/kaggle/input/cityscapes/Cityscapes_v2.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  annotation= torch.tensor(self.transform(annotation),dtype=torch.uint8)


Epoch: 1
- Train Acc: 0.451
- Train mIou: 0.088

- Test Acc: 0.247
- Test Loss: 107.279
- Test mIoU: 0.070
__________________________
Epoch: 2
- Train Acc: 0.479
- Train mIou: 0.104

- Test Acc: 0.444
- Test Loss: 77.282
- Test mIoU: 0.099
__________________________
Epoch: 3
- Train Acc: 0.491
- Train mIou: 0.111

- Test Acc: 0.441
- Test Loss: 90.574
- Test mIoU: 0.074
__________________________
Epoch: 4
- Train Acc: 0.494
- Train mIou: 0.111

- Test Acc: 0.409
- Test Loss: 86.769
- Test mIoU: 0.070
__________________________
Epoch: 5
- Train Acc: 0.494
- Train mIou: 0.113

- Test Acc: 0.462
- Test Loss: 70.677
- Test mIoU: 0.102
__________________________
Epoch: 6
- Train Acc: 0.505
- Train mIou: 0.119

- Test Acc: 0.437
- Test Loss: 142.351
- Test mIoU: 0.071
__________________________
Epoch: 7
- Train Acc: 0.508
- Train mIou: 0.122

- Test Acc: 0.394
- Test Loss: 100.639
- Test mIoU: 0.067
__________________________
Epoch: 8
- Train Acc: 0.507
- Train mIou: 0.120

- Test Acc: 0.431

In [14]:
#Plot graphs
metrics = np.stack(arrays=[train_iou_list ,train_acc_list , train_loss_list, test_iou_list, test_acc_list, test_loss_list], axis=0)

names = ["mIou", "Accuracy", "Loss"]
plotted = 0
plt.figure(figsize=(15,3))
for i in range(len(names)):
    plotted += 1
    plt.subplot(1,3,plotted) 
    plt.plot(range(1, 51), metrics[i], label="Train")
    plt.plot(range(1, 51), metrics[i+len(names)], label="Test")
    plt.title(f"{names[i]}")
    plt.legend()
plt.show()

In [112]:
import torch.nn as nn
plot_loader = DataLoader(cityscapes_val, batch_size=1, shuffle=True)
img, sem_map = next(iter(plot_loader))
img, sem_map = img.to(device), sem_map.to(device)
img = img.float()
sem_map = sem_map.int()
outputs = model(img)
softmax = nn.Softmax(dim=1)
prediction = torch.argmax(softmax(outputs),axis=1)

img_transpose = img.cpu().detach().numpy().squeeze(0).transpose(1,2,0)
map_transpose = sem_map.cpu().detach().numpy().squeeze(0).transpose(1,2,0)
out_map = prediction.cpu().detach().numpy().transpose(1,2,0)

print(f"Image shape: {np.shape(img_transpose)}")
print(f"Semantic Map shape: {np.shape(map_transpose)}")
print(f"Predicted Map shape: {np.shape(out_map)}")

Image shape: (512, 1024, 3)
Semantic Map shape: (512, 1024, 1)
Predicted Map shape: (512, 1024, 1)


/kaggle/input/cityscapes/Cityscapes_v2.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  annotation= torch.tensor(self.transform(annotation),dtype=torch.uint8)


In [113]:
plt.figure(figsize=(10,10))
plt.subplot(3,1,1)
plt.imshow(img_transpose)
plt.title("Image")
plt.subplot(3,1,2)
plt.imshow(map_transpose, cmap="Blues")
plt.title("Semantic Map")
plt.subplot(3,1,3)
plt.imshow(out_map, cmap="Blues")
plt.title("Predicted Map")
plt.show()